## Prerequisites {#prerequisites}

Make sure you have the following packages installed:

In [ ]:
pip install pydantic openai pydantic-ai

You also need access to the OpenAI API with a valid key:

In [ ]:
export OPENAI_API_KEY="your-api-key"

## Core Workflow: Building a Type-Safe Agent {#core-workflow-building-a-type-safe-agent}

First, define a `Pydantic` Model that describes the expected structure of your agent's output:

In [ ]:
from pydantic import BaseModel
from typing import List

class ApplicantProfile(BaseModel):
    first_name: str
    last_name: str
    experience_years: int
    primary_skill: List[str]

This model acts as a contract, ensuring that the language model returns a structured object with the correct fields and types.

Now, use the `output_type` parameter to connect this model to your agent:

In [ ]:
from pydantic_ai import Agent

agent = Agent(
    'gpt-4o-mini-2024-07-18',
    system_prompt='Extract name, years of experience, and primary skill from the job applicant description.',
    output_type=ApplicantProfile,
)

result = agent.run_sync('Khuyen Tran is a data scientist with 5 years of experience, skilled in Python and machine learning.')
print(result.output)

Output:

```python
first_name='Khuyen' last_name='Tran' experience_years=5 primary_skill=['Python', 'machine learning']
```

This structured output is safe to pass directly into downstream applications without modification.

`result.output` returns a Pydantic object. To convert it into a standard Python dictionary for further use, call:

In [ ]:
result.output.model_dump()

Output:

```json
{
  "first_name": "Khuyen",
  "last_name": "Tran",
  "experience_years": 5,
  "primary_skill": [
    "Python",
    "machine learning"
  ]
}
```

You can now easily integrate this into other data workflows. For example, to convert the output into a pandas DataFrame:

In [ ]:
import pandas as pd

df = pd.DataFrame([result.output.model_dump()])
df

Output:

```python
  first_name last_name  experience_years     primary_skill
0     Khuyen      Tran                 5            Python
1     Khuyen      Tran                 5  machine learning
```



## Using the DuckDuckGo Search Tool {#using-the-duckduckgo-search-tool}

Have you ever tried to make your AI app respond to current events or user queries with real-world data without managing a custom search backend?

PydanticAI supports integrating tools like [DuckDuckGo search](https://ai.pydantic.dev/common-tools/#duckduckgo-search-tool) to enhance your AI agents with live web results.

In [ ]:
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool
from typing import List

class UnemploymentDataSource(BaseModel):
    title: List[str]
    description: List[str]
    url: List[str]

# Define the agent with DuckDuckGo search tool
search_agent = Agent(
    'gpt-4o-mini-2024-07-18',
    tools=[duckduckgo_search_tool()],
    system_prompt='Search DuckDuckGo and return links or resources that match the query.',
    output_type=UnemploymentDataSource,
)

# Run a search for unemployment rate dataset
unemployment_result = search_agent.run_sync(
    'Monthly unemployment rate dataset for US from 2018 to 2024'
)

print(unemployment_result.output)

Example output:

```python
title=[
  'Civilian unemployment rate - U.S. Bureau of Labor Statistics',
  'Databases, Tables & Calculators by Subject - U.S. Bureau of Labor Statistics',
  'Unemployment Rate (UNRATE) | FRED | St. Louis Fed',
  'US Unemployment Rate Monthly Analysis: Employment Situation - YCharts',
  'U.S. Unemployment Rate 1991-2025 - Macrotrends'
]
description=[
  'The U.S. Bureau of Labor Statistics provides information on the civilian unemployment rate.',
  'Access various data tables and calculators related to employment situations in the U.S.',
  "Access historical unemployment rates and data through the St. Louis Fed's FRED database.",
  'In-depth view into historical data of the U.S. unemployment rate including projections.',
  'Details on U.S. unemployment rate trends and statistics from 1991 to 2025.'
]
url=[
  'https://www.bls.gov/charts/employment-situation/civilian-unemployment-rate.htm',
  'https://www.bls.gov/data/',
  'https://fred.stlouisfed.org/series/UNRATE/',
  'https://ycharts.com/indicators/us_unemployment_rate',
  'https://www.macrotrends.net/global-metrics/countries/USA/united-states/unemployment-rate'
]
```

This output is fully structured and aligns with the `UnemploymentDataSource` schema. It makes the data easy to load into tables or use in downstream analytics workflows without additional transformation.

## Comparison with LangChain Structured Output {#comparison-with-langchain-structured-output}

### How PydanticAI Handles Structured Output {#how-pydanticai-handles-structured-output}

PydanticAI returns Pydantic objects directly, so you can immediately access structured fields like `cook_time` without extra parsing.

In [ ]:
from typing import Optional, List
from pydantic import BaseModel
from pydantic_ai import Agent

class RecipeExtractor(BaseModel):
    ingredients: List[str]
    instructions: str
    cook_time: Optional[str]

recipe_agent = Agent(
    "gpt-4o-mini-2024-07-18",
    system_prompt="Pull ingredients, instructions, and cook time.",
    output_type=RecipeExtractor,
)

recipe_result = recipe_agent.run_sync(
    "Sugar, flour, cocoa, eggs, and milk. Mix, bake at 350F for 30 min."
)
print(recipe_result.output.cook_time)
# 30 minutes

PydanticAI simplifies standalone LLM tasks, meaning tasks where you prompt a model once and immediately use the structured output without needing multiple steps, chaining, or external orchestration.

### How LangChain Handles Structured Output {#how-langchain-handles-structured-output}

[LangChain](https://www.langchain.com/) binds a Pydantic model to the tool, but you must manually extract values from `tool_calls`, adding an extra step.

In [ ]:
from typing import Optional, List
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize the chat model
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)

# Bind the response formatter schema
model_with_tools = model.bind_tools([RecipeExtractor])

# Create a list of messages to send to the model
messages = [
    SystemMessage("Pull ingredients, instructions, and cook time."),
    HumanMessage("Sugar, flour, cocoa, eggs, and milk. Mix, bake at 350F for 30 min."),
]

# Invoke the model with the prepared messages
ai_msg = model_with_tools.invoke(messages)

# Access the tool calls made during the model invocation
print(ai_msg.tool_calls[0]['args']['cook_time'])
# 30 minutes

LangChain is better suited for multi-step workflows, such as combining several tools, using routing logic, or building custom chains.